In [1]:
from pycalphad import Database, Model, calculate, equilibrium, variables as v
from xarray import DataArray

In [2]:
class PrecipitateModel(Model):
    matrix_chempots = []
    @property
    def matrix_hyperplane(self):
        return sum(self.moles(self.nonvacant_elements[i])*self.matrix_chempots[i]
                   for i in range(len(self.nonvacant_elements)))
    @property
    def GM(self):
        return self.ast - self.matrix_hyperplane

class GibbsThompsonModel(Model):
    "Spherical particle."
    radius = 1e-6 # m
    volume = 7.3e-6 # m^3/mol
    interfacial_energy = 250e-3 # J/m^2
    elastic_misfit_energy = 0 # J/mol
    @property
    def GM(self):
        return self.ast + (2*self.interfacial_energy/self.radius + self.elastic_misfit_energy) * self.volume

In [3]:
import numpy as np

def parallel_tangent(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp):
    conds = {v.N: 1, v.P: 1e5, v.T: temp}
    conds.update(matrix_comp)
    matrix_eq = equilibrium(dbf, comps, matrix_phase, conds)
    # pycalphad currently doesn't have a way to turn global minimization off and directly specify starting points
    if matrix_eq.isel(vertex=1).Phase.values.flatten() != ['']:
        raise ValueError('Matrix phase has miscibility gap. This bug will be fixed in the future')
    matrix_chempots = matrix_eq.MU.values.flatten()
    # This part will not work until mass balance constraint can be relaxed
    #precip = PrecipitateModel(dbf, comps, precipitate_phase)
    #precip.matrix_chempots = matrix_chempots
    #conds = {v.N: 1, v.P: 1e5, v.T: temp}
    #df_eq = equilibrium(dbf, comps, precipitate_phase, conds, model=precip)
    df_eq = calculate(dbf, comps, precipitate_phase, T=temp, N=1, P=1e5)
    df_eq['GM'] = df_eq.X.values[0,0,0].dot(matrix_chempots) - df_eq.GM
    selected_idx = df_eq.GM.argmax()
    return matrix_eq.isel(vertex=0), df_eq.isel(points=selected_idx)

def nucleation_barrier(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp,
                       interfacial_energy, precipitate_volume):
    "Spherical precipitate."
    matrix_eq, precip_eq = parallel_tangent(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp)
    precip_driving_force = float(precip_eq.GM.values) # J/mol
    elastic_misfit_energy = 0 # J/m^3
    barrier = 16./3 * np.pi * interfacial_energy **3 / (precip_driving_force + elastic_misfit_energy) ** 2 # J/mol
    critical_radius = 2 * interfacial_energy / ((precip_driving_force / precipitate_volume) + elastic_misfit_energy) # m
    print(temp, critical_radius)
    if critical_radius < 0:
        barrier = np.inf
    cluster_area = 4*np.pi*critical_radius**2
    cluster_volume = (4./3) * np.pi * critical_radius**3
    return barrier, precip_driving_force, cluster_area, cluster_volume, matrix_eq, precip_eq

def growth_rate(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp, particle_radius):
    conds = {v.N: 1, v.P: 1e5, v.T: temp}
    conds.update(matrix_comp)
    # Fictive; Could retrieve from TDB in principle
    mobility = 1e-7 * np.exp(-14e4/(8.3145*temp)) # m^2 / s
    mobilities = np.eye(len(matrix_comp)+1) * mobility

    matrix_ff_eq, precip_eq = parallel_tangent(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp)
    precip_driving_force = float(precip_eq.GM.values) # J/mol
    interfacial_energy = 250e-3 # J/m^2
    precipitate_volume = 7.3e-6 # m^3/mol
    elastic_misfit_energy = 0 # J/m^3
    # Spherical particle
    critical_radius = 2 * interfacial_energy / ((precip_driving_force / precipitate_volume) + elastic_misfit_energy) # m
    # XXX: Should really be done with global min off, fixed-phase conditions, etc.
    # As written, this will break with miscibility gaps
    particle_mod = GibbsThompsonModel(dbf, comps, precipitate_phase)
    particle_mod.radius = particle_radius
    interface_eq = equilibrium(dbf, comps, [matrix_phase, precipitate_phase], conds,
                               model={precipitate_phase: particle_mod})
    matrix_idx = np.nonzero((interface_eq.Phase==matrix_phase).values.flatten())[0]
    if len(matrix_idx) > 1:
        raise ValueError('Matrix phase has miscibility gap')
    elif len(matrix_idx) == 0:
        # Matrix is metastable at this composition; massive transformation kinetics?
        print(interface_eq)
        raise ValueError('Matrix phase is not stable')
    else:
        matrix_idx = matrix_idx[0]
        matrix_interface_eq = interface_eq.isel(vertex=matrix_idx)

    precip_idx = np.nonzero((interface_eq.Phase==precipitate_phase).values.flatten())[0]
    if len(precip_idx) > 1:
        raise ValueError('Precipitate phase has miscibility gap')
    elif len(precip_idx) == 0:
        precip_conc = np.zeros(len(interface_eq.component))
        # Precipitate is metastable at this radius (it will start to dissolve)
        # Compute equilibrium for precipitate by itself at parallel tangent composition
        pt_comp = {v.X(str(comp)): precip_eq.X.values.flatten()[idx] for idx, comp in enumerate(precip_eq.component.values[:-1])}
        conds = {v.N: 1, v.P: 1e5, v.T: temp}
        conds.update(pt_comp)
        precip_interface_eq = equilibrium(dbf, comps, precipitate_phase, conds,
                               model={precipitate_phase: particle_mod})
        precip_interface_eq = precip_interface_eq.isel(vertex=0)
    else:
        precip_idx = precip_idx[0]
        precip_interface_eq = interface_eq.isel(vertex=precip_idx)
    
    eta_geo_factor = 1.0
    # Growth equation from Eq. 12, M. Paliwal and I.-H Jung, Calphad, 2019
    velocity = 2*interfacial_energy * precipitate_volume * (1./critical_radius - 1./particle_radius)
    x_int_precip = precip_interface_eq.X.values.flatten()
    x_int_matrix = matrix_interface_eq.X.values.flatten()
    denominator = 0
    for i in range(mobilities.shape[0]-1):
        for j in range(mobilities.shape[1]-1):
            denominator += (x_int_precip[i] - x_int_matrix[i]) * (x_int_precip[j] - x_int_matrix[j]) / mobilities[i,j]
    velocity /= denominator
    return velocity



def precipitate_size_distribution(dbf, comps, matrix_phase, initial_matrix_comp, precipitate_phase, temp):
    interfacial_energy = 250e-3 # J/m^2
    precipitate_volume = 7.3e-6 # m^3/mol
    elastic_misfit_energy = 0 # J/m^3
    atomic_distance = 2e-10 # m
    n_prefactor = 1e18
    # Could retrieve from TDB in principle
    diffusivity = 1e-7 * np.exp(-14e4/(8.3145*temp)) # m^2 / s
    delta_t = 100
    time_steps = int(1e4)
    precipitate_dist = np.zeros((time_steps, 100+1)) # Extra fictive size class for finite-difference simplicity
    size_classes = np.logspace(-10, -8, num=100)
    # Compositions at interface
    matrix_composition = np.zeros((time_steps, len(initial_matrix_comp)+1))
    precipitate_composition = np.zeros((time_steps, len(initial_matrix_comp)+1))

    g_star, g_nucl, area, volume, matrix_eq, precip_eq = nucleation_barrier(dbf, comps, matrix_phase,
                                                                            initial_matrix_comp, precipitate_phase,
                                                                            temp, interfacial_energy,
                                                                            precipitate_volume)
    matrix_composition[0, :] = matrix_eq.X.values.flatten()
    precipitate_composition[0, :] = precip_eq.X.values.flatten()

    for step in range(time_steps):
        if step > 0:
            matrix_comp = {v.X(c): matrix_composition[step][comp_idx] for comp_idx, c in enumerate(matrix_eq.component.values[:-1])}
            g_star, g_nucl, area, volume, matrix_eq, precip_eq = nucleation_barrier(dbf, comps, matrix_phase,
                                                                                    matrix_comp, precipitate_phase,
                                                                                    temp, interfacial_energy,
                                                                                    precipitate_volume)
            matrix_composition[step, :] = matrix_eq.X.values.flatten()
            precipitate_composition[step, :] = precip_eq.X.values.flatten()

        print('Step ', step, 'Matrix Comp: ', matrix_composition[step], 'Precipitate Comp:', precipitate_composition[step])
        precip_driving_force = float(precip_eq.GM.values) # J/mol
        elastic_misfit_energy = 0 # J/m^3
        critical_radius = 2 * interfacial_energy / ((precip_driving_force / precipitate_volume) + elastic_misfit_energy) # m

        zeldovich = np.sqrt(interfacial_energy/(8.3145*temp)) * precipitate_volume / (2*np.pi*6.022e23 * critical_radius**2) # dim-less

        conc_diff = (precip_eq.X.values.flatten() - matrix_eq.X.values.flatten()) ** 2 /\
                    (matrix_eq.X.values.flatten() * diffusivity)
        beta_attachment_rate = (area / ((atomic_distance**4) * precipitate_volume)) / conc_diff.sum()  # s^-1
        incubation_time = 1 / (2*beta_attachment_rate*zeldovich) # s

        # Now we know the nucleation rate, and the critical radius
        # XXX: Missing the incubation contribution
        ss_nucl_rate = n_prefactor * zeldovich * beta_attachment_rate * np.exp(-g_star/(8.3145*temp))
        print('ss_nucl_rate', ss_nucl_rate)
        if critical_radius < 0:
            critical_radius = 0.0

        active_size_class_idx = np.argmax(size_classes > critical_radius)
        if critical_radius > size_classes[-1]:
            active_size_class_idx = size_classes.shape[0] - 1
        elif critical_radius < size_classes[0]:
            active_size_class_idx = 0
        if active_size_class_idx > 0:
            precipitate_dist[step, active_size_class_idx - 1] += ss_nucl_rate * delta_t

        # Compute growth rates for all size classes
        velocities = np.zeros(precipitate_dist.shape[1])
        matrix_comp = {v.X(c): matrix_composition[step][comp_idx] for comp_idx, c in enumerate(matrix_eq.component.values[:-1])}
        for idx in range(size_classes.shape[0]):
            radius = size_classes[idx]
            velocities[idx] = growth_rate(dbf, comps, matrix_phase, matrix_comp, precipitate_phase, temp, radius)
        # Explicit finite difference method with first-order upwinding scheme
        if velocities[0] > 0:
            precipitate_dist[step+1, 0] = precipitate_dist[step, 0] - (delta_t / (size_classes[1] - size_classes[0])) * velocities[0] * precipitate_dist[step, 0]
        else:
            precipitate_dist[step+1, 0] = precipitate_dist[step, 0] - (delta_t / (size_classes[1] - size_classes[0])) * velocities[0] * precipitate_dist[step, 1]
        for i in range(1, size_classes.shape[0]):
            delta_r = size_classes[i] - size_classes[i-1]
            if (velocities[i-1] > 0) and velocities[i] > 0: 
                precipitate_dist[step+1, i] += (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i-1] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i]
            elif (velocities[i-1] > 0) and velocities[i] < 0: 
                precipitate_dist[step+1, i] += (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i-1] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i+1]
            elif (velocities[i-1] < 0) and velocities[i] > 0: 
                precipitate_dist[step+1, i] += (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i]
            else:
                precipitate_dist[step+1, i] += (delta_t / delta_r) * velocities[i-1] * precipitate_dist[step, i] - (delta_t / delta_r) * velocities[i] * precipitate_dist[step, i+1]

        np.clip(precipitate_dist, 0., None, out=precipitate_dist)
        # Update volume fraction of precipitate (spherical assumption)
        total_precipitate_volume = np.dot((4/3) * np.pi * size_classes ** 3, precipitate_dist[step+1,:-1]) # m^3
        print('total_precipitate_volume', total_precipitate_volume)
        matrix_domain_volume = precipitate_volume * 1 # because N=1; assume equal for simplicity; m^3
        precip_volume_fraction = total_precipitate_volume / matrix_domain_volume
        print('precip_volume_fraction', precip_volume_fraction)
        if step == time_steps - 1:
            break
        # Update matrix composition
        matrix_composition[step+1, :] = (matrix_composition[0 , :] - precip_volume_fraction * precipitate_composition[step]) / (1 - precip_volume_fraction)
    return matrix_composition, precipitate_composition, precipitate_dist

In [4]:
dbf = Database('Al-Cu-Zr_Zhou.tdb')
comps = ['AL', 'CU', 'VA']

results = precipitate_size_distribution(dbf, comps, 'BCC_A2', {v.X('CU'): 0.6}, 'ETA2', 800)
matrix_composition = results[0]
precipitate_composition = results[1]
precipitate_dist = results[2]

800 3.621579765132555e-09
Step  0 Matrix Comp:  [0.4 0.6] Precipitate Comp: [0.48 0.52]
ss_nucl_rate 3.4503650633334796e+16
total_precipitate_volume 2.3736435731213924e-07
precip_volume_fraction 0.03251566538522455
800 3.80284401959202e-09
Step  1 Matrix Comp:  [0.39731132 0.60268868] Precipitate Comp: [0.48 0.52]
ss_nucl_rate 3.2222977179882236e+16
total_precipitate_volume 4.535578905699684e-07
precip_volume_fraction 0.06213121788629704
800 4.005146725128374e-09
Step  2 Matrix Comp:  [0.39470022 0.60529978] Precipitate Comp: [0.48 0.52]
ss_nucl_rate 3.0211747838076456e+16
total_precipitate_volume 7.739433972317548e-07
precip_volume_fraction 0.10601964345640476
800 4.398824305738999e-09
Step  3 Matrix Comp:  [0.39051258 0.60948742] Precipitate Comp: [0.48 0.52]
ss_nucl_rate 2.7346989962154468e+16
total_precipitate_volume 1.5074057725663948e-06
precip_volume_fraction 0.20649394144745134
800 6.239473806289259e-09
Step  4 Matrix Comp:  [0.37918161 0.62081839] Precipitate Comp: [0.48 0.52]

ValueError: Matrix phase is not stable

In [ ]:
matrix_composition

In [ ]:
equilibrium(dbf, comps, ['BCC_A2', 'ETA2'], {v.N:1, v.T: 800, v.P: 1e5, v.X('CU'): 0.6})

In [ ]:
from pycalphad import calculate
import matplotlib.pyplot as plt
from pycalphad.plot.utils import phase_legend
import numpy as np

my_phases = ['ETA2', 'BCC_A2']
legend_handles, colorlist = phase_legend(my_phases)

fig = plt.figure(figsize=(9,6))
ax = fig.gca()
for name in my_phases:
    result = calculate(dbf, ['AL', 'CU', 'VA'], name, P=101325, T=300, output='GM')
    ax.scatter(result.X.sel(component='CU'), result.GM,
               marker='.', s=5, color=colorlist[name.upper()])
ax.set_xlim((0, 1))
ax.legend(handles=legend_handles, loc='center left', bbox_to_anchor=(1, 0.6))
plt.show()

In [ ]:
dbf.phases